In [ ]:
eps = 300

# 1 
# image = io.imread('wc_cv/cv/plots/WC+8Co_5_crack.jpg')[200:-300][:400]
# 2
image = io.imread('wc_cv/cv/plots/WC+8Co_5_fixed_cropped.jpg')
# 3
# image = io.imread('wc_cv/datasets/original/o_bc_left/Ultra_Co6_2/Ultra_Co6_2-003.jpeg')[:300,:500]
# 4
# image = io.imread('wc_cv/datasets/original/o_bc_left/Ultra_Co6_2/Ultra_Co6_2-001.jpeg')[:200,:500]

border=30

(entry_nodes,
 exit_nodes,
 img_contours_o,
 img_preprocessed_final,
 cnts,
 nodes_metadata)  = Crack.preprocess_graph_image(image, border=border, disk=5,entry_ellps_w=5,exit_ellps_w=5,r=4)

# plot=False
plot=True

if plot:
    fontsize=15
    # fig, axes = plt.subplots(1,2,figsize=(20,20))
    # axes[0].imshow(img_contours_o)
    # axes[1].imshow(img_preprocessed_final)

    # axes[0].set_xlabel('width, pixels',fontsize=fontsize)
    # axes[0].set_ylabel('height, pixels',fontsize=fontsize)

    # axes[1].set_xlabel('width, pixels',fontsize=fontsize)
    # axes[1].set_ylabel('height, pixels',fontsize=fontsize)
    plt.show()

    # plt.imshow(img_contours_o)
    # plt.xlabel('width, pixels',fontsize=fontsize)
    # plt.ylabel('height, pixels',fontsize=fontsize)
    # # plt.savefig('wc_cv/cv/Ultra_Co6_2-001_cut_step_1.jpeg',bbox_inches='tight')
    # plt.show()

    # plt.imshow(img_preprocessed_final)
    # plt.xlabel('width, pixels',fontsize=fontsize)
    # plt.ylabel('height, pixels',fontsize=fontsize)
    # # plt.savefig('wc_cv/cv/Ultra_Co6_2-001_cut_step_2.jpeg',bbox_inches='tight')
    # plt.show()

g, img_contours =  Crack.create_crack_graph(img_preprocessed_final.shape, cnts, nodes_metadata, eps=eps)

In [ ]:
entry_nodes = np.unique(paths['entry_node'])
exit_nodes = np.unique(paths['exit_node'])

shortest_entry_paths = []
for entry_node in tqdm(entry_nodes):
    row = paths[paths.entry_node==entry_node].sort_values(by='path_len_pixels').iloc[0]
    shortest_entry_paths.append(row)

shortest_exit_paths = []
for exit_node in tqdm(exit_nodes):
    row = paths[paths.exit_node==exit_node].sort_values(by='path_len_pixels').iloc[0]
    shortest_exit_paths.append(row)

df_shortest_entry = pd.DataFrame(shortest_entry_paths)
df_shortest_exit = pd.DataFrame(shortest_exit_paths)

In [ ]:

df_shortest_entry['path'] = df_shortest_entry['path'].apply(lambda x: np.array(x))
df_shortest_entry.to_csv('wc_cv/cv/df_shortest_entry.csv',index=False)

df_shortest_exit['path'] = df_shortest_exit['path'].apply(lambda x: np.array(x))
df_shortest_exit.to_csv('wc_cv/cv/df_shortest_exit.csv',index=False)


In [ ]:
import networkx as nx
from graph_tool.all import Graph

def nx_to_gt_with_attrs(nx_graph):
    gt_graph = Graph(directed=nx_graph.is_directed())
    vprop_name = gt_graph.new_vertex_property("string")
    eprop_weight = gt_graph.new_edge_property("float")

    nx_to_gt_v = {}
    for node in nx_graph.nodes(data=True):
        v = gt_graph.add_vertex()
        nx_to_gt_v[node[0]] = v
        if 'name' in node[1]:
            vprop_name[v] = str(node[1]['name'])

    for u, v, data in nx_graph.edges(data=True):
        e = gt_graph.add_edge(nx_to_gt_v[u], nx_to_gt_v[v])
        if 'weight' in data:
            eprop_weight[e] = float(data['weight'])

    gt_graph.vertex_properties['name'] = vprop_name
    gt_graph.edge_properties['weight'] = eprop_weight

    return gt_graph

gt_graph = nx_to_gt_with_attrs(g_weighed)

gt.graph_draw(gt_graph, vertex_text=gt_graph.vertex_index, output="wc_cv/cv/two-nodes.png")

# eweight = gt_graph.new_edge_property("weight")
eweight = gt_graph.new_edge_property("float")

vlist = gt_graph.get_vertices()
edges = gt_graph.get_edges()

paths =  list(gt.all_shortest_paths(gt_graph, source=entry_node, target=exit_node, weights=eweight))

# eweight = gt_graph.new_edge_property("weight")
eweight = gt_graph.new_edge_property("float")

vlist = gt_graph.get_vertices()
edges = gt_graph.get_edges()

paths =  list(gt.all_shortest_paths(gt_graph, source=entry_node, target=exit_node, weights=eweight))